# Selection in pandas using ```query```.

Now for something a little different. 

This is the sixth post in a series on indexing and selecting in pandas, and so far we have covered concepts about indexing and selecting data in both pandas ```Series``` and ```DataFrame```s using concepts like slicing, boolean indexing, and familiar python concepts like callables. This post will cover the details behind the ```query``` method and requires a little more background and explanation. In the end, you'll see how it's useful and how it can help speed up your calculations.

Before we dive in, if you are jumping in the middle and want to get caught up, here's what has been discussed so far:

 * [Basic indexing, selecting by label and location](https://www.wrighters.io/indexing-and-selecting-in-pandas-part-1/)
 * [Slicing in pandas](https://www.wrighters.io/indexing-and-selecting-in-pandas-slicing/)
 * [Selecting by boolean indexing](https://www.wrighters.io/boolean-indexing-in-pandas/)
 * [Selecting by callable](https://www.wrighters.io/indexing-and-selecting-in-pandas-by-callable/)
 * [Selecting using where and mask](https://www.wrighters.io/selecting-in-pandas-using-where-and-mask/)


In pandas, ```DataFrame```s have a ```query``` method that supports selection using an expression as a string. One way to think about this (and a natural way to think about it due to its name and basic functionality) is that it can sort of be like SQL. But I'm not sure that's a great way to think about it, for a couple of reasons. First, it is limited in the language it supports (i.e. don't count on writing queries with complex joins or group by expressions). Second, the purpose of this expression is not just to give you a simple query language for selecting data, but to speed up expressions.

That being said, using query does offer a number of advantages, with a few complications, which we will cover.

Let's start with a few examples of what an expression in ```query``` might look like.

In [1]:
import pandas as pd
import numpy as np

# create a sample dataframe of floating points around 0
df = pd.DataFrame(np.random.random((5,5)) - .5, columns=list("abcde"))

In [2]:
df

,a,b,c,d,e
0,0.367389,0.018962,-0.106435,0.397229,-0.237226
1,0.016208,-0.151215,-0.465317,-0.295572,-0.153494
2,0.391743,-0.013441,-0.125635,0.137689,-0.152479
3,-0.488314,-0.303783,0.457164,0.447945,-0.054434
4,-0.475356,-0.095161,-0.484103,-0.055500,0.035111


### This is still boolean indexing
If you look back on the [post on boolean indexing](https://www.wrighters.io/boolean-indexing-in-pandas/), you remember that to select rows in a ```DataFrame``` using any of the indexing methods, you can pass in a boolean vector the same size as the ```DataFrame``` index. So, if we want to select all rows in our ```DataFrame``` where the value in column ```b``` is negative, we pass in a boolean array created by applying the less than (```<```) operation to the column.

In [3]:
df[df["b"] < 0]

,a,b,c,d,e
1,0.016208,-0.151215,-0.465317,-0.295572,-0.153494
2,0.391743,-0.013441,-0.125635,0.137689,-0.152479
3,-0.488314,-0.303783,0.457164,0.447945,-0.054434
4,-0.475356,-0.095161,-0.484103,-0.055500,0.035111


The ```query``` method is similar. It takes an expression, which is evaluated in the context of the ```DataFrame```, and that expression returns a boolean value used for selection. So we can write the same selection like this:

In [4]:
df.query("b < 0")

,a,b,c,d,e
1,0.016208,-0.151215,-0.465317,-0.295572,-0.153494
2,0.391743,-0.013441,-0.125635,0.137689,-0.152479
3,-0.488314,-0.303783,0.457164,0.447945,-0.054434
4,-0.475356,-0.095161,-0.484103,-0.055500,0.035111


More complex expressions can be expressed much simpler. For example, to select rows where column ```b``` is less than column ```c``` *and* column ```c``` is less than column ```e```, we need to use a fair amount of parentheses and repeat our ```df``` variable in order to have it parse properly.

In [5]:
df[(df["b"] < df["c"]) & (df["c"] < df["e"])]

,a,b,c,d,e


Using ```query```, this can be written in a much more elegant way. For combining and modifying expressions, you can use ```&``` and ```|``` and ```~```, or ```and``` and ```or``` and ```not```.

In [6]:
df.query("(b < c) & (c < e)")
df.query("(b < c) and (c < e)")

,a,b,c,d,e


Or even better

In [7]:
df.query("b < c < e")

,a,b,c,d,e


You can also use your index in the query, by using ```index```, or you can rename your index and use that.

In [8]:
df.query("index > 2")
df.index.name = 'idx'
df.query("idx > 2")

,a,b,c,d,e
idx,,,,,
3,-0.488314,-0.303783,0.457164,0.447945,-0.054434
4,-0.475356,-0.095161,-0.484103,-0.055500,0.035111


### Variables from your environment
You can also use variables that are in scope inside your expression by prefixing them with an ```@```.

In [9]:
limit = 0.1
df.query("b > @limit")

,a,b,c,d,e
idx,,,,,


If your columns are not valid Python variable names, you'll have to surround them with backticks (Note you'll need pandas 0.25 or higher for this functionality, and 1.0+ for expanded support).

In [10]:
df['mean value'] = df.mean(axis=1)
df.query('`mean value` > d')

,a,b,c,d,e,mean value
idx,,,,,,
1,0.016208,-0.151215,-0.465317,-0.295572,-0.153494,-0.209878


The ```in``` operator is supported in query, so you can use that as well, either using a variable, or existing columns, or as literals.  The ```===``` and ```!=``` operators work like ```in``` and ```not in``` for lists.

In [11]:
df['name'] = ['x', 'y', 'z', 'p', 'q']
search = ['x', 'y']
df.query("name in ['x', 'y']")
# or
df.query('name in @search')
# or
df.query('name == @search')
# or
df.query("name == ['x', 'y']")

,a,b,c,d,e,mean value,name
idx,,,,,,,
0,0.367389,0.018962,-0.106435,0.397229,-0.237226,0.087984,x
1,0.016208,-0.151215,-0.465317,-0.295572,-0.153494,-0.209878,y


### What about updates?
This expression is handy, but what about updating your ```DataFrame```? I know we've mostly talked about selecting data, but it turns out that you can also update or add columns by using expressions, but with the ```eval``` method. Note that ```eval``` returns a copy of the modified data, so you need to assign back to your variable or use the ```inplace``` argument to retain the changes.

In [12]:
df.eval('x = a * b')

,a,b,c,d,e,mean value,name,x
idx,,,,,,,,
0,0.367389,0.018962,-0.106435,0.397229,-0.237226,0.087984,x,0.006966
1,0.016208,-0.151215,-0.465317,-0.295572,-0.153494,-0.209878,y,-0.002451
2,0.391743,-0.013441,-0.125635,0.137689,-0.152479,0.047575,z,-0.005265
3,-0.488314,-0.303783,0.457164,0.447945,-0.054434,0.011716,p,0.148342
4,-0.475356,-0.095161,-0.484103,-0.055500,0.035111,-0.215002,q,0.045235


Note you can pass in multiple expressions to be evaluated in one go.

In [13]:
df.eval('''
x = a * b
y = c * d
''')

,a,b,c,d,e,mean value,name,x,y
idx,,,,,,,,,
0,0.367389,0.018962,-0.106435,0.397229,-0.237226,0.087984,x,0.006966,-0.042279
1,0.016208,-0.151215,-0.465317,-0.295572,-0.153494,-0.209878,y,-0.002451,0.137535
2,0.391743,-0.013441,-0.125635,0.137689,-0.152479,0.047575,z,-0.005265,-0.017299
3,-0.488314,-0.303783,0.457164,0.447945,-0.054434,0.011716,p,0.148342,0.204784
4,-0.475356,-0.095161,-0.484103,-0.055500,0.035111,-0.215002,q,0.045235,0.026868


### What about updates with multiple values?
It turns out there is a also a higher level pandas ```eval``` function available for doing more complicated updates involving multiple ```DataFrames``` or ```Series```. I won't get it into it in much detail here, but you can use it for the same sorts of expressions as above.

In [14]:
s = pd.Series(np.random.random(5))
pd.eval('g = df.a + s', target=df)

,a,b,c,d,e,mean value,name,g
idx,,,,,,,,
0,0.367389,0.018962,-0.106435,0.397229,-0.237226,0.087984,x,0.786829
1,0.016208,-0.151215,-0.465317,-0.295572,-0.153494,-0.209878,y,0.740401
2,0.391743,-0.013441,-0.125635,0.137689,-0.152479,0.047575,z,0.537689
3,-0.488314,-0.303783,0.457164,0.447945,-0.054434,0.011716,p,-0.269873
4,-0.475356,-0.095161,-0.484103,-0.055500,0.035111,-0.215002,q,-0.358582


### One other nice use case
One more nice advantage of using ```query``` (and ```eval```) is that you may have a situation where you have multiple ```DataFrame```s in your code that share column names where you'd like to use the same expression on them. If you didn't use ```query```, you'd have to pass these into a function so you refer to these in local python expressions. So as these expressions get more complex, ```query``` may make sense for code clarity.

In [15]:
df2 = pd.DataFrame(np.random.random((5,5)) - .5, columns=list("abcde"))

# without query, need to build expression in python
def find_d_gt_e(d):
    return d[d['d'] > d['e']]

find_d_gt_e(df)
find_d_gt_e(df2)

expr = "d > e" # same expression, re-usable anywhere these columns exist

df.query(expr)
df2.query(expr);

## But what's the point?
So there are a few advantages to the mini-langage used by ```query``` and ```eval```, resulting in cleaner looking code. But there are still some disadvantages to using a differing query language. First, since you're passing in a string, there's no syntax checking being done by your editor. You may find this annoying or it may make you slightly less efficient. Also, you have to learn a new set of rules about what expressions are allowed and even possible in this mini language.

### It's mainly about speed
What is not entirely obvious here is that under the hood you can install a nice library called ```numexpr```  ([docs](https://numexpr.readthedocs.io/en/latest/index.html), [src](https://github.com/pydata/numexpr)) that exists to make calculations with large NumPy (and pandas) objects potentially much faster. When you use ```query``` or ```eval```, this expression is passed into ```numexpr``` and optimized using its bag of tricks. Expected performance improvement can be between .95x and up to 20x, with average performance around 3-4x for typical use cases. You can read details in the docs, but essentially ```numexpr``` takes vectorized operations and makes them work in chunks that optimize for cache and CPU branch prediction. If your arrays are really large, your cache will not be hit much, and if you break your large arrays into very small pieces, your CPU won't be as efficient. 

Make sure you install it in your environment first if it's not there.

In [16]:
%pip install numexpr

You should consider upgrading via the '/Users/mcw/.pyenv/versions/3.8.6/envs/pandas/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Here's a simple NumPy example from the ```numexpr``` docs that shows what the improvement can look like.

In [17]:
import numexpr as ne

a = np.random.rand(int(1e6))
b = np.random.rand(int(1e6))

r1 = %timeit -o 2 * a + 3 * b
r2 = %timeit -o ne.evaluate("2 * a + 3 * b")

5.2 ms ± 425 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.06 ms ± 83.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
print(f"{r1.average / r2.average:.2f}x faster")

2.52x faster


You can see much more detail in the ```numexpr``` docs if you're interested, but for a pandas specific example, let's see what the difference is when using numexpr and not for some ```DataFrames``` in the size range where we can start to expect an improvement (over 200k rows).

In [19]:
sd = pd.DataFrame(np.random.random(int(3e7)).reshape(int(1e7), 3))
sd2 = pd.DataFrame(np.random.random(int(3e7)).reshape(int(1e7), 3))

r1 = %timeit -o 3 * sd + 4 * sd2

118 ms ± 4.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
r2 = %timeit -o pd.eval('3 * sd + 4 * sd2')

112 ms ± 4.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
print(f"{r1.average / r2.average: .2f}x faster")

 1.05x faster


Depending on what you are doing, some operations will be sped up even more.

In [22]:
r1 = %timeit -o 3 * sd + sd2 ** 4

260 ms ± 22.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
r2 = %timeit -o pd.eval('3 * sd + sd2 ** 4')

85.4 ms ± 2.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
print(f"{r1.average / r2.average: .2f}x faster")

 3.04x faster


### Wrapping up
When you're working with initial exploration of data, or smaller data sets, using ```query``` and ```eval``` will probably not be your go-to options for data selection. But consider these methods for speeding up code that needs to work with large data sets or is run repeatedly, it can make a big difference. It's also useful for cleaning up code with complicated selection criteria.


I also think this is a good place to wrap up the series on selecting and indexing in pandas. Now that the basics have been covered, there are many more topics worth looking at.